# Notebook 1 - Ingestão Bronze
Neste notebook vamos coletar todos os dados que utilizaremos na nossa análise e armazená-los aqui no Databricks.
Os dados foram coletados de sites oficiais e armazenados, inicialmente, no GitHub. O Notebook:
- Baixa os arquivos do GitHub;
- Transforma em parquet;
- Faz uma amostra dos dados de criminalidade (3,6 milhões de registros).

## Introdução - Sobre os dados que vamos usar

Neste trabalho vamos processar os dados abertos de crimes do estado de São Paulo, dos anos de 2022, 2023 e 2024, disponibilizados no portal de transparência do estado, complementados pelo dicionário de dados fornecido pela Secretaria de Segurança Pública do estado de São Paulo.

Os dados completos podem ser encontrados na aba de [estatísticas](https://www.ssp.sp.gov.br/estatistica) do portal de transparência, mais especificamente na parte de [consultas](https://www.ssp.sp.gov.br/estatistica/consultas).

Quanto ao dicionário de dados, como eu já tinha a intenção de analisar esses dados e por esses dados serem oficiais do estado de São Paulo, solicitei no portal [Fala SP](https://fala.sp.gov.br/) informações sobre todos os atributos contidos neles.

Este é o dicionário:

|Atributos|Descrição|
|---|---|
|ANO\_BO|Ano do Boletim|
|ANO\_ESTATISTICA|Ano em que a ocorrência foi inserida na estatística oficial |
|BAIRRO|Bairro da Ocorrência|
|CIDADE|Cidade de Registro|
|DATA\_COMUNICACAO|Data da Comunicação|
|DATA\_OCORRENCIA\_BO|Data da Ocorrência|
|DESC\_PERIODO|Período da Ocorrência|
|DESCR\_CONDUTA|Tipo de local ou circunstancia que qualifica a ocorrencia|
|DESCR\_TIPOLOCAL|Descreve grupo de tipos de locais onde se deu o fato|
|HORA\_OCORRENCIA\_BO|Hora da Ocorrência|
|LATITUDE|Latitude da Ocorrência|
|LOGRADOURO|Logradouro dos fatos|
|LONGITUDE|Longitude da Ocorrência|
|MÊS ESTATISTICA|Mês em que a ocorrência foi inserida na estatística oficial |
|NATUREZA\_APURADA|Natureza de Publicação|
|NOME\_DELEGACIA|Delegacia responsável pelo registro|
|NOME\_DELEGACIA\_CIRCUNSCRIÇÃO|Delegacia de Circunscrição|
|NOME\_DEPARTAMENTO|Departamento responsável pelo registro|
|NOME\_DEPARTAMENTO\_CIRCUNSCRIÇÃO|Departamento de Circunscrição|
|NOME\_MUNICIPIO\_CIRCUNSCRIÇÃO|Município da Delegacia de Circunscrição|
|NOME\_SECCIONAL|Delegacia Seccional responsável pelo registro|
|NOME\_SECCIONAL\_CIRCUNSCRIÇÃO|Seccional de Circunscrição|
|NUM\_BO|Número do Boletim|
|NUMERO\_LOGRADOURO|Numero do Logradouro dos fatos|
|RUBRICA|Natureza juridica da ocorrencia|


Além desses dados, vamos utilizar a base do IBGE de população do estado de São Paulo, os dados podem ser encontrados no Portal do IBGE [Cidades e Estados](https://www.ibge.gov.br/cidades-e-estados) mais especificamente na parte de [São Paulo](https://www.ibge.gov.br/cidades-e-estados/sp.html).

## 1. Setup Inicial
Importanto as bibliotecas necessárias.

In [0]:
import urllib.request
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, count, round

import os

## 2. Coletando os Dados
Coletando as bases de dados salvas no repositório público no GitHub.

In [0]:
anos = [2022, 2023, 2024]
meses = [f"{m:02d}" for m in range(1, 13)]
base_url = "https://raw.githubusercontent.com/nathpignaton/mvp_criminalidade_sp/main/dados/bronze"

# Criar pastas no DBFS
for ano in anos:
    dbutils.fs.mkdirs(f"dbfs:/FileStore/mvp_criminalidade_sp/{ano}")

# Baixar para /tmp e mover para o DBFS
for ano in anos:
    aa = str(ano)[2:]  # '22', '23', '24'
    
    for mm in meses:
        nome_arquivo = f"{aa}{mm}SPDC.csv"
        url = f"{base_url}/{ano}/{nome_arquivo}"
        local_tmp_path = f"/tmp/{nome_arquivo}"
        dbfs_path = f"dbfs:/FileStore/mvp_criminalidade_sp/{ano}/{nome_arquivo}"
        
        try:
            urllib.request.urlretrieve(url, local_tmp_path)
            dbutils.fs.cp(f"file:{local_tmp_path}", dbfs_path)
        except Exception as e:
            print(f"Erro ao baixar {nome_arquivo}: {e}")

Unindo os arquivos baixados.

In [0]:
df_union: DataFrame = None

for ano in anos:
    caminho = f"/FileStore/mvp_criminalidade_sp/{ano}/*.csv"
    df_temp = spark.read.option("header", True).option("inferSchema", True).csv(caminho)
    df_union = df_temp if df_union is None else df_union.union(df_temp)
    print(f"Arquivos de {ano} lidos.")

df_union.printSchema()
df_union.show(5)

Arquivos de 2022 lidos.
Arquivos de 2023 lidos.
Arquivos de 2024 lidos.
root
 |-- NOME_DEPARTAMENTO: string (nullable = true)
 |-- NOME_SECCIONAL: string (nullable = true)
 |-- NOME_DELEGACIA: string (nullable = true)
 |-- CIDADE: string (nullable = true)
 |-- NUM_BO: string (nullable = true)
 |-- ANO_BO: integer (nullable = true)
 |-- DATA_COMUNICACAO_BO: timestamp (nullable = true)
 |-- DATA_OCORRENCIA_BO: string (nullable = true)
 |-- HORA_OCORRENCIA_BO: timestamp (nullable = true)
 |-- DESCR_PERIODO: string (nullable = true)
 |-- DESCR_TIPOLOCAL: string (nullable = true)
 |-- BAIRRO: string (nullable = true)
 |-- LOGRADOURO: string (nullable = true)
 |-- NUMERO_LOGRADOURO: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- NOME_DELEGACIA_CIRCUNSCRIÇÃO: string (nullable = true)
 |-- NOME_DEPARTAMENTO_CIRCUNSCRIÇÃO: string (nullable = true)
 |-- NOME_SECCIONAL_CIRCUNSCRIÇÃO: string (nullable = true)
 |-- NOME_MUNICIPIO_CIRCU

Criando o parquet completo com todos os dados de criminalidade.

In [0]:
# não vou rodar essa célula para não substituir o arquivo sem necessidade.
df_union.write.mode("overwrite").parquet("/FileStore/mvp_criminalidade_sp/bronze/full_bronze.parquet")

Criando uma amostra com 0,1% dos registros, para análises teste.

In [0]:
# Remover registros com valores nulos nas colunas importantes
df_clean = df_union.filter(
    (col("NATUREZA_APURADA").isNotNull()) &
    (col("NOME_MUNICIPIO_CIRCUNSCRI\u00c7\u00c3O").isNotNull())
)

# Aplicar amostragem aleatória global (0,1% dos registros)
df_amostra = df_clean.sample(withReplacement=False, fraction=0.001, seed=42)

# Visualizar os dados amostrados
df_amostra.show(5)

+--------------------+--------------------+--------------------+--------------------+-------+------+-------------------+-------------------+-------------------+-------------+---------------+--------------+--------------------+-----------------+-----------------+-----------------+----------------------------+-------------------------------+----------------------------+----------------------------+--------------------+-------------+----------------+---------------+---------------+
|   NOME_DEPARTAMENTO|      NOME_SECCIONAL|      NOME_DELEGACIA|              CIDADE| NUM_BO|ANO_BO|DATA_COMUNICACAO_BO| DATA_OCORRENCIA_BO| HORA_OCORRENCIA_BO|DESCR_PERIODO|DESCR_TIPOLOCAL|        BAIRRO|          LOGRADOURO|NUMERO_LOGRADOURO|         LATITUDE|        LONGITUDE|NOME_DELEGACIA_CIRCUNSCRIÇÃO|NOME_DEPARTAMENTO_CIRCUNSCRIÇÃO|NOME_SECCIONAL_CIRCUNSCRIÇÃO|NOME_MUNICIPIO_CIRCUNSCRIÇÃO|             RUBRICA|DESCR_CONDUTA|NATUREZA_APURADA|MES_ESTATISTICA|ANO_ESTATISTICA|
+--------------------+----------

Checando a representatividade da amostra.

In [0]:
# Calcular totais
total_pop = df_clean.count()
total_amostra = df_amostra.count()

# Distribuição por NATUREZA_APURADA
dist_pop_natureza = df_clean.groupBy("NATUREZA_APURADA") \
    .agg(count("*").alias("qtd_pop")) \
    .withColumn("perc_pop", round(col("qtd_pop") / total_pop * 100, 2))

dist_amostra_natureza = df_amostra.groupBy("NATUREZA_APURADA") \
    .agg(count("*").alias("qtd_amostra")) \
    .withColumn("perc_amostra", round(col("qtd_amostra") / total_amostra * 100, 2))

# Distribuição por MUNICÍPIO (com escape de caracteres especiais)
col_municipio = "NOME_MUNICIPIO_CIRCUNSCRI\u00C7\u00C3O"

dist_pop_cidade = df_clean.groupBy(col(col_municipio)) \
    .agg(count("*").alias("qtd_pop")) \
    .withColumn("perc_pop", round(col("qtd_pop") / total_pop * 100, 2)) \
    .withColumnRenamed(col_municipio, "MUNICIPIO")

dist_amostra_cidade = df_amostra.groupBy(col(col_municipio)) \
    .agg(count("*").alias("qtd_amostra")) \
    .withColumn("perc_amostra", round(col("qtd_amostra") / total_amostra * 100, 2)) \
    .withColumnRenamed(col_municipio, "MUNICIPIO")

# Comparando distribuições
comparativo_natureza = dist_pop_natureza.join(
    dist_amostra_natureza,
    on="NATUREZA_APURADA",
    how="outer"
).fillna(0).orderBy(col("perc_pop").desc())

print("Comparativo por NATUREZA_APURADA")
comparativo_natureza.show(truncate=False)

comparativo_cidade = dist_pop_cidade.join(
    dist_amostra_cidade,
    on="MUNICIPIO",
    how="outer"
).fillna(0).orderBy(col("perc_pop").desc())

print("Comparativo por MUNICÍPIO")
comparativo_cidade.show(truncate=False)


Comparativo por NATUREZA_APURADA
+-----------------------------------------------+-------+--------+-----------+------------+
|NATUREZA_APURADA                               |qtd_pop|perc_pop|qtd_amostra|perc_amostra|
+-----------------------------------------------+-------+--------+-----------+------------+
|FURTO - OUTROS                                 |1693081|46.62   |1653       |45.09       |
|ROUBO - OUTROS                                 |647544 |17.83   |662        |18.06       |
|LESÃO CORPORAL DOLOSA                          |432580 |11.91   |441        |12.03       |
|FURTO DE VEÍCULO                               |284900 |7.84    |299        |8.16        |
|LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO|203782 |5.61    |236        |6.44        |
|ROUBO DE VEÍCULO                               |110883 |3.05    |112        |3.06        |
|TRÁFICO DE ENTORPECENTES                       |73535  |2.02    |85         |2.32        |
|TRAFICO DE ENTORPECENTES                      

Como pudemos ver, a amostra está representativa e, por isso, vamos poder usar essa amostra para testar as análises antes de realizar no banco inteiro.

In [0]:
# não vou rodar essa célula novamente para não substituir o arquivo sem necessidade
# salvando a amostra
caminho_amostra = "/FileStore/mvp_criminalidade_sp/bronze/amostra_representativa.parquet"

# salvando a amostra como Parquet
df_amostra.write.mode("overwrite").parquet(caminho_amostra)

print(f"Amostra representativa salva com sucesso em: {caminho_amostra}")


Amostra representativa salva com sucesso em: /FileStore/mvp_criminalidade_sp/bronze/amostra_representativa.parquet


## 3. Coletando dados complementares
Por fim, vamos coletar a base do IBGE, que também está no repositório público no GitHub.

In [0]:
# URL direta do arquivo no GitHub
url_ibge = "https://raw.githubusercontent.com/nathpignaton/mvp_criminalidade_sp/main/dados/bronze/ibge_sp.csv"

# Caminhos locais
local_tmp_path = "/tmp/ibge_estado_sp.csv"
dbfs_path = "dbfs:/FileStore/mvp_criminalidade_sp/bronze/ibge_sp.csv"

# Download + cópia pro DBFS
try:
    urllib.request.urlretrieve(url_ibge, local_tmp_path)
    dbutils.fs.cp(f"file:{local_tmp_path}", dbfs_path)
    print("Arquivo IBGE carregado com sucesso.")
except Exception as e:
    print(f"Erro ao baixar arquivo IBGE: {e}")


Arquivo IBGE carregado com sucesso.
